# Best practices for writing Dockerfiles


# Minimize RUN instruction layers


- Several instructions add a layer to the image each time the Docker engine executes them. And each layer adds extra size to the final image and can affect the overall performance. One of those instructions is the RUN instruction. To avoid this unwanted increase in size and decrease in performance, you can combine several instructions into one. Let's check this in practice using the following Dockerfile:

In [ ]:
FROM ubuntu:22.04

LABEL author=HyperUser

RUN apt-get update -y \
  && apt-get upgrade -y \
  && apt-get install iputils-ping -y \
  && apt-get install net-tools -y

ENTRYPOINT ["/bin/bash"]


```Try to avoid the apt-get upgrade command. It can bring upgrades that can affect the environment you use.```

Let's build an ubuntu:v1 image based on it and modify it to have separate RUN instructions for each command and build an image again.

In [ ]:
FROM ubuntu:22.04

LABEL author=HyperUser

RUN apt-get update -y 
RUN apt-get upgrade -y
RUN apt-get install iputils-ping -y 
RUN apt-get install net-tools -y

ENTRYPOINT ["/bin/bash"]

!!! not a good practice to have multiple seperate runs !!

Now, let's examine what picture the docker images -a command shows us.



```
$ docker images --format='{{.Repository}}\t{{.Tag}}\t{{.Size}}' | head -2
ubuntu       v2        108MB
ubuntu       v1        107MB
```

# Choosing the right base image


The basis for any image is its base image. So far, you should have come across examples where Ubuntu is used for this purpose. Ubuntu is known for its user-friendly approach, but its image includes many packages that you may not need. If there is no need to use this particular image, you can switch to another Linux distribution image: the Alpine Linux image. It's based on BusyBox and requires less space because it includes a minimal set of packages and tools. Let's create images with minimal configurations using both distributions and compare their size. First, let's use Ubuntu:

In [ ]:
FROM ubuntu:22.04

LABEL author=HyperUser

ENTRYPOINT ["echo", "Hello, Students."]

After this, let's change the base image to Alpine:



In [ ]:
FROM alpine:3.17

LABEL author=HyperUser

ENTRYPOINT ["echo", "Hello, Students."]

Now, let's check what we've got. This is what you'll see when you list your images:



```
$ docker images --format='{{.Repository}}\t{{.Tag}}\t{{.Size}}' | head -2
alpine  v1      7.05MB
ubuntu  v1      70.2MB
```

<font color='yellow' size=25># Removing unnecessary packages </font>

One of the challenges while using Docker is trying to reduce the image size as much as possible. A good practice to reduce the image size is removing apt lists after installing something. Let's work with a Dockerfile used to install network tools. The ubuntu:v1 image size built on it was 107MB. This time, let's apply the same Dockerfile but remove apt lists at the end.



In [ ]:
FROM ubuntu:22.04

LABEL author=HyperUser

RUN apt-get update -y \
  && apt-get upgrade -y \
  && apt-get install iputils-ping -y \
  && apt-get install net-tools -y \
  && rm -rf /var/lib/apt/lists/*

ENTRYPOINT ["/bin/bash"]

This time, the image size is much smaller:
```
$ docker images 
REPOSITORY   TAG       IMAGE ID       CREATED              SIZE
ubuntu       v1        1141a699133b   About a minute ago   72.2MB
```

Another important practice is using the ```--no-install-recommends``` flag when installing any package. Many Linux distributions, such as Debian or Ubuntu, use the idea of required and recommended packages. 

In [ ]:
FROM ubuntu:22.04

LABEL author=HyperUser

RUN apt-get update -y \
  && apt-get upgrade -y \
  && apt-get install iputils-ping -y --no-install-recommends \
  && apt-get install net-tools -y --no-install-recommends \
  && rm -rf /var/lib/apt/lists/*

ENTRYPOINT ["/bin/bash"]

<font color='yellow' size=25>Using the exec form</font>


```
# shell form
CMD echo Hello, Students.
ENTRYPOINT echo Hello, Students.

# exec form
CMD ["echo", "Hello, World."]
ENTRYPOINT ["echo", "Hello, World."]

# exec form, combination of CMD and ENTRYPOINT
ENTRYPOINT ["echo"]
CMD ["Hello"]
```

Both forms have their advantages and disadvantages. For instance, you can choose the shell form if you need shell processing. If you have an application that must accept different parameters each time you run the container, ENTRYPOINT with exec form can be a good choice.

However, exec is the recommended form. It wins over the shell form due to its performance advantage. exec doesn't start a shell process. It only invokes the required command with arguments and without any other process. Using this form is also preferred from a security perspective as you execute only the specified command.

<font color='yellow' size=25> Using the linter 

Creating a Dockerfile by adhering to best practices is a valuable skill. The image's size, building time, and security level depend on how smartly you create your Dockerfile. Fortunately, there is a tool called Hadolint static analyzer that helps you find issues in your Dockerfile and improve it by showing hints. You can either download it from the official GitHub repo or use its online version. To check how it works, let's take the previous Dockerfile sample and try to find its issues.

```
FROM ubuntu:22.04

LABEL author=HyperUser

RUN apt-get update -y \
  && apt-get upgrade -y \
  && apt-get install iputils-ping -y \
  && apt-get install net-tools -y

ENTRYPOINT ["/bin/bash"]
```

<font color='yellow' size=25> Auditing the image for security vulnerabilities


Auditing Docker images for security vulnerabilities is another essential part of working with images. For this purpose, you can apply many techniques, like checking the base image layers or analyzing the image for any unnecessary packages or dependencies. On top of this, Docker provides us with a docker scout command to scan the Dockerfile and image to automatically detect issues. Starting with version 4.17, docker scout is automatically installed when you install Docker Desktop. However, to manually install you can run the following commands:

In [ ]:
curl -fsSL https://raw.githubusercontent.com/docker/scout-cli/main/install.sh -o install-scout.sh
sh install-scout.sh

Finally, log in to Docker Hub using Docker Desktop or CLI using the ```docker login -u <username>``` command. When you are all set up, run the command docker scout quickview or docker scout cves to scan for vulnerabilities in local or remote images. To scan a local image, run ```docker scout quickview <image name>```.

docker scout quickview provides only an overview of vulnerabilities whereas docker scout cves provides a full list of vulnerabilities.

# The export command


In [ ]:
export NEW_VARIABLE="The export command"


In [ ]:
NEW_VARIABLE="The export command"
export NEW_VARIABLE